In [21]:
!pip install imageio matplotlib numpy imageio_ffmpeg

Defaulting to user installation because normal site-packages is not writeable


In [23]:
import json
import matplotlib.pyplot as plt
import numpy as np
import math

def rotate_points(x, y, angle):
    """Rotates points by the given angle."""
    x_rot = x * np.cos(angle) - y * np.sin(angle)
    y_rot = x * np.sin(angle) + y * np.cos(angle)
    return x_rot, y_rot

def is_inside_ellipse(x, y, h, k, a, b, theta):
    """
    Check if a point (x, y) is inside the rotated ellipse.
    
    Args:
    x, y: Coordinates of the point to check
    h, k: Center of the ellipse
    a, b: Semi-major and semi-minor axes of the ellipse
    theta: Rotation angle of the ellipse in radians
    
    Returns:
    bool: True if the point is inside or on the ellipse, False otherwise
    """
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    x_translated = x - h
    y_translated = y - k
    
    x_rotated = cos_theta * x_translated + sin_theta * y_translated
    y_rotated = -sin_theta * x_translated + cos_theta * y_translated
    
    return (x_rotated**2 / a**2) + (y_rotated**2 / b**2) <= 1

def draw_fov(ax, tilt_deg, rotation_deg, base_radius, satellites, connected_satellite):
    """Draws the FOV ellipse with given tilt, rotation, and base radius, and adds satellite points."""
    # Plot settings
    ax.set_ylim(0, 90)
    ax.set_yticks(np.arange(0, 91, 10))
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)
    ax.grid(True)

    # FOV ellipse and axes
    center_shift = tilt_deg
    x_radius = base_radius
    y_radius = math.sqrt(base_radius**2 - tilt_deg**2)

    theta = np.linspace(0, 2 * np.pi, 300)
    x = x_radius * np.cos(theta) + center_shift
    y = y_radius * np.sin(theta)
    r = np.sqrt(x**2 + y**2)
    angles = np.arctan2(y, x) + np.deg2rad(rotation_deg)
    ax.plot(angles, r, 'r', label=f'FOV (Base Radius: {base_radius})')

    # Plot major and minor axes
    major_axis_x = np.array([center_shift + x_radius, center_shift - x_radius])
    major_axis_y = np.array([0, 0])
    minor_axis_x = np.array([center_shift, center_shift])
    minor_axis_y = np.array([y_radius, -y_radius])
    major_axis_x_rot, major_axis_y_rot = rotate_points(major_axis_x, major_axis_y, np.deg2rad(rotation_deg))
    minor_axis_x_rot, minor_axis_y_rot = rotate_points(minor_axis_x, minor_axis_y, np.deg2rad(rotation_deg))
    ax.plot(np.arctan2(major_axis_y_rot, major_axis_x_rot), np.sqrt(major_axis_x_rot**2 + major_axis_y_rot**2), 'red', linestyle='--', linewidth=1)
    ax.plot(np.arctan2(minor_axis_y_rot, minor_axis_x_rot), np.sqrt(minor_axis_x_rot**2 + minor_axis_y_rot**2), 'red', linestyle='--', linewidth=1)

    # Calculate the center point of the FOV (intersection of major and minor axes)
    fov_center_x = center_shift
    fov_center_y = 0
    fov_center_x_rot, fov_center_y_rot = rotate_points(fov_center_x, fov_center_y, np.deg2rad(rotation_deg))
    fov_center_r = np.sqrt(fov_center_x_rot**2 + fov_center_y_rot**2)
    fov_center_theta = np.arctan2(fov_center_y_rot, fov_center_x_rot)

    # Scatter satellite positions with labels
    for sat in satellites:
        elevation = sat['altitude_deg']
        azimuth = sat['azimuth_deg']
        scatter_r = 90 - elevation  # Polar plot r (90 - Elevation degrees)
        scatter_theta = np.radians(azimuth)  # Polar plot theta (Azimuth in radians)
        
        # Check if the satellite is inside the FOV
        is_inside = is_inside_ellipse(scatter_r * np.cos(scatter_theta), scatter_r * np.sin(scatter_theta),
                                      center_shift, 0, x_radius, y_radius, np.deg2rad(rotation_deg))
        
        color = 'green' if is_inside else 'red'
        ax.scatter(scatter_theta, scatter_r, color=color, s=10)
        
        # Add label for each satellite
        satellite_name = sat['satellite'].split('-')[-1]  # Extract the number part of the satellite name
        ax.annotate(satellite_name, (scatter_theta, scatter_r), xytext=(3, 3), 
                    textcoords='offset points', fontsize=8, alpha=0.7)
        
        # Draw green line to connected satellite
        if satellite_name == connected_satellite:
            # Satellite polar position
            satellite_r = 90 - elevation  # Correct radius for satellite position
            satellite_theta = np.radians(azimuth)  # Correct angle for satellite position
            
            # Line from the center of the FOV to the satellite position
            ax.plot([fov_center_theta, satellite_theta], [fov_center_r, satellite_r], color='green', linewidth=2, alpha=0.7)

    ax.legend(loc='upper right')

# Load JSON data
with open("output/satellite_visibility.json", "r") as f:
    data = json.load(f)

# Select a timestamp (you can loop over them too)
timestamp = list(data.keys())[0]
satellites = data[timestamp]
tilt = data[timestamp][0]['tilt_deg']
rotation = data[timestamp][0]['rotation_deg']
# Create plot
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': 'polar'})

# Draw FOV with connected satellite
connected_satellite = "5185"  # For example, using first satellite
draw_fov(ax, tilt_deg=tilt, rotation_deg=rotation, base_radius=50, satellites=satellites, connected_satellite=connected_satellite)

ax.set_title(f"Satellite FOVs at {timestamp}")
plt.show()


KeyError: 0

In [24]:
import imageio
import os
import matplotlib.pyplot as plt

def generate_video_from_satellites(data, output_video="satellite_fov.mp4", tilt_deg=27, rotation_deg=-1.3, base_radius=60, fps=2):
    # Load serving satellite data
    serving_satellite_df = pd.read_csv("output/combined_serving_satellite_data.csv")
    serving_satellite_df['Timestamp'] = pd.to_datetime(serving_satellite_df['Timestamp'])
    
    # Temporary directory to save images
    os.makedirs("frames", exist_ok=True)
    frame_files = []
    connected_satellite = ["5185", "5507", "5507", "3629", "3072", "32697", "4208"]
    for i, (timestamp, satellites) in enumerate(data.items()):
        # Find the connected satellite at this timestamp
        timestamp_dt = pd.to_datetime(timestamp)
        connected_satellite = serving_satellite_df.loc[serving_satellite_df['Timestamp'].nearest(timestamp_dt), 'Connected_Satellite'].iloc[0]
        connected_satellite = connected_satellite.split('-')[-1]  # Extract the number part
        
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': 'polar'})
        draw_fov(ax, tilt_deg=tilt_deg, rotation_deg=rotation_deg, base_radius=base_radius, satellites=satellites, connected_satellite=connected_satellite)
        ax.set_title(f"Satellite FOVs at {timestamp}")
        
        # Save frame
        filename = f"frames/frame_{i:03d}.png"
        plt.savefig(filename)
        frame_files.append(filename)
        plt.close(fig)

    # Create video
    with imageio.get_writer(output_video, mode='I', fps=fps) as writer:
        for filename in frame_files:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Clean up frame files
    for f in frame_files:
        os.remove(f)
    os.rmdir("frames")

    return output_video

In [25]:
generate_video_from_satellites(data, output_video="satellite_fov.mp4", fps=2)

FileNotFoundError: [Errno 2] No such file or directory: 'output/combined_serving_satellite_data.csv'